## Read and process reflectances per paintout

In [ ]:
import pandas as pd
from pathlib import Path

# TODO: update this path to the location of the dataset on your system
dataset_path = Path('/local/home/rwiersma/Documents/Paintings/Captures paintings 2023')

# Read the overview csv file that lists the paintouts, their captures, and the mixing ratios
with (dataset_path / 'paintout_measurements/paintouts.csv').open() as file:
    paintouts = pd.read_csv(file)

paintout_names = paintouts['name'].unique()

# Print list of names of paintouts
print(paintout_names)

['Titanium white' 'Bone black' 'Chalk white' 'French Ochre 40030'
 'French Ochre JALS 40060' 'Lapis Lazuli from chile' 'Raw umber greenish'
 'Red lake' 'Indigo' 'Jaune de reseda' 'Krapplack red']


In [ ]:
import numpy as np
import spectral.io.envi as envi
from skimage.io import imread
from scipy.interpolate import interp1d

import matplotlib.pyplot as plt

from painting_tools.util.color import spectral_to_rgb, resample_wavelengths

# Read out first file for wavelengths
reflectance_path = dataset_path / 'paintout_measurements' / '855' / 'results' / f'REFLECTANCE_855'
# Reads out calibrated reflectance data (raw data and white/black reference are also available)
metadata = envi.open(str(reflectance_path) + '.hdr', str(reflectance_path) + '.dat').metadata

# Create an empty pandas dataframe
columns = ['name', 'ratio_pigment_to_white', 'background', *metadata['wavelength']]
reflectance_df = pd.DataFrame(columns=columns)

# Flag to visualize lookup
visualize_lookup = False

# Iterate over all samples in the paintout and get the reflectance data from the masked regions
for i, row in paintouts.iterrows(): 
    capture_folder = dataset_path / 'paintout_measurements' / str(row['capture_id'])
    reflectance_path = capture_folder / 'results' / f'REFLECTANCE_{row['capture_id']}'

    # Reads out calibrated reflectance data (raw data and white/black reference are also available)
    reflectance_data = envi.open(str(reflectance_path) + '.hdr', str(reflectance_path) + '.dat')
    reflectance = reflectance_data.asarray()
    reflectance = np.rot90(reflectance, -1)
    wavelengths = np.array(reflectance_data.metadata['wavelength'], dtype=np.float32)

    # Get the pixel_mask for this capture
    # This mask was painted by hand to select samples of the paintout without highlights or paper covering the paint
    pixel_mask = imread(capture_folder / f'sample_locations_{row['capture_id']}.tif')

    # Take average of all pixels in reflectance where pixel_mask == pixel_id
    reflectance_white = reflectance[pixel_mask == row['pixel_id_white']].mean(axis=0)
    reflectance_black = reflectance[pixel_mask == row['pixel_id_black']].mean(axis=0)

    # Add the reflectance data to the dataframe
    if reflectance_df.empty:
        reflectance_df = pd.DataFrame([[row['name'], row['ratio_pigment_to_white'], 'white', *reflectance_white]], columns=columns)
    else:
        reflectance_df = pd.concat([reflectance_df, pd.DataFrame([[row['name'], row['ratio_pigment_to_white'], 'white', *reflectance_white]], columns=columns)], ignore_index=True)
    reflectance_df = pd.concat([reflectance_df, pd.DataFrame([[row['name'], row['ratio_pigment_to_white'], 'black', *reflectance_black]], columns=columns)], ignore_index=True)

    if visualize_lookup and i < 3:
        # For visualization, convert to RGB, including gamma correction
        reflectance_rgb = (np.array(spectral_to_rgb(reflectance, wavelengths, apply_gamma=True, gamma=2.2)) * 255).astype(np.uint8)

        fig, ax = plt.subplots(1, 2, figsize=(10, 5))
        ax[0].imshow(reflectance_rgb)
        ax[0].imshow(np.ones((512, 512)), alpha=1.0*(pixel_mask == row['pixel_id_white']), cmap='grey', vmin=0, vmax=1)
        ax[0].imshow(np.zeros((512, 512)), alpha=1.0*(pixel_mask == row['pixel_id_black']), cmap='grey', vmin=0, vmax=1)

        # Plot reflectance
        plt.title(f'{row['name']} {row['ratio_pigment_to_white']}')
        ax[1].plot(wavelengths, reflectance_white, label='white')
        ax[1].plot(wavelengths, reflectance_black, label='black')
        ax[1].legend()
        plt.show()

# Save the reflectance data to a csv file
reflectance_df.to_csv(dataset_path / 'processed' / 'paintouts_reflectance.csv', index=False)

: 